In [2]:
%%time
import pandas as pd
import json
import os

# Define keywords for filtering
keywords = {
    "Camera": ["Camera", "Cameras", "CCTV", "Security Camera", "Webcam", "IP Camera", 
               "Surveillance Camera", "Network Camera", "Video Camera", "Dome Camera", 
               "Bullet Camera", "PTZ Camera", "Digital Camera", "Analog Camera"],
    "Router": ["Router", "Routers", "Wireless Router", "WiFi Router", "Gateway", 
               "Modem Router", "Broadband Router", "DSL Router", "Edge Router", 
               "Core Router", "Home Router", "Enterprise Router"],
    "Switch": ["Switch", "Switches", "Network Switch", "Ethernet Switch", "LAN Switch", 
               "Managed Switch", "Unmanaged Switch", "Gigabit Switch", "PoE Switch", 
               "Industrial Switch", "Layer 2 Switch", "Layer 3 Switch", "Smart Switch"],
    "NVR": ["NVR", "NVRs", "Network Video Recorder", "Video Recorder", 
            "Digital Video Recorder", "DVR", "Surveillance Recorder", 
            "Security Recorder", "IP Video Recorder", "CCTV Recorder"]
}
# Function to get relevant tags based on the description or product
def get_relevant_tags(data, keywords):
    # Safely extract the description and product fields
    containers = data.get('containers', {}).get('cna', {})
    
    # Ensure 'descriptions' and 'affected' are lists and non-empty
    description_data = containers.get('descriptions', [])
    if isinstance(description_data, list) and len(description_data) > 0:
        description = description_data[0].get('value', '').lower()
    else:
        description = ''

    affected_data = containers.get('affected', [])
    if isinstance(affected_data, list) and len(affected_data) > 0:
        product = affected_data[0].get('product', '').lower()
    else:
        product = ''

    # Check for matching keywords in the description and product
    for maintype, subtypes in keywords.items():
        for subtype in subtypes:
            if subtype.lower() in description or subtype.lower() in product:
                return maintype, subtype
    return None, None


# Path for the filtered data CSV file
filtered_data_csv = './data/filtered_cve_data2.csv'

# Check if the filtered data CSV already exists
if not os.path.exists(filtered_data_csv):
    # Set the base directory containing the CVE files
    base_dir = './cvelistV5/cves'

    # List to hold the filtered CVE data for DataFrame
    filtered_data_list = []

    # Walk through the directory structure
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.json') and file not in ['delta.json', 'deltalog.json']:
                file_path = os.path.join(root, file)
                
                # Read and load the JSON file with UTF-8 encoding
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                    # Get relevant tags based on description or product
                    maintype, subtype = get_relevant_tags(data, keywords)
                    
                    if maintype and subtype:
                        # Extract CVE fields
                        cve_id = file[:-5]
                        description = data.get('containers', {}).get('cna', {}).get('descriptions', [{}])[0].get('value', 'No description available')
                        affected = data.get('containers', {}).get('cna', {}).get('affected', [{}])[0]
                        product = affected.get('product', 'N/A')
                        vendor = affected.get('vendor', 'N/A')
                        versions_info = affected.get('versions', [])
                        
 
                        # Extract version details
                        device_version_list = [v.get('version', 'N/A').lower() for v in versions_info if v.get('version')]
                        if not device_version_list:
                            for v in data.get('containers', {}).get('cna', {}).get('affected', []):
                                device_version = v.get('version')
                                if device_version:
                                    device_version_list.append(device_version)
                        device_version = ', '.join(device_version_list) if device_version_list else 'n/a'

                        firmware = affected.get('platforms', 'N/A')
                        patch_availability = "Yes" if any("patch" in ref.get('tags', []) for ref in data.get('containers', {}).get('cna', {}).get('references', [])) else 'N/A'
                        reserved_date = data.get('cveMetadata', {}).get('dateReserved', 'N/A')
                        published_date = data.get('cveMetadata', {}).get('datePublished', 'N/A')
                        update_date = data.get('cveMetadata', {}).get('dateUpdated', 'N/A')
 
                        # Add this entry to the list
                        filtered_data_list.append({
                            'CVE_ID': cve_id,
                            'Description': description,
                            'Device':maintype,
                            'Product': product,
                            'Vendor': vendor,
                            'Version': device_version,
                            'Firmware': firmware,
                            'Patch Availability': patch_availability,
                            'Reserved Date': reserved_date,
                            'Published Date': published_date,
                            'Update Date': update_date
                        })

    # Create DataFrame and save it to CSV
    filtered_df = pd.DataFrame(filtered_data_list)
    filtered_df.to_csv(filtered_data_csv, index=False)
    print("Filtered CVE data CSV created.")

else:
    # Load the existing filtered CSV
    filtered_df = pd.read_csv(filtered_data_csv)
    print("Filtered CVE data CSV loaded.")

# Check for required fields and create them if missing
required_fields = ['CVE_ID', 'Description','Device' ,'Product', 'Vendor', 'Version', 'Firmware', 'Patch Availability','Problem Type',
                   'Reserved Date', 'Published Date', 'Update Date', 'EPSS_Score', 'CVSS_Score', 'Severity', 
                   'Vector', 'Exploitability Score', 'Impact Score', 'Score Source', 'references']

# Iterate through each required field
for field in required_fields:
    if field not in filtered_df.columns:
        filtered_df[field] = 'N/A'
        print(f"Field '{field}' added to the DataFrame.")

# Save the updated DataFrame if any fields were added
filtered_df.to_csv(filtered_data_csv, index=False)
print("Data verification and field check completed.")


Filtered CVE data CSV created.
Field 'Problem Type' added to the DataFrame.
Field 'EPSS_Score' added to the DataFrame.
Field 'CVSS_Score' added to the DataFrame.
Field 'Severity' added to the DataFrame.
Field 'Vector' added to the DataFrame.
Field 'Exploitability Score' added to the DataFrame.
Field 'Impact Score' added to the DataFrame.
Field 'Score Source' added to the DataFrame.
Field 'references' added to the DataFrame.
Data verification and field check completed.
CPU times: user 9.79 s, sys: 1.36 s, total: 11.1 s
Wall time: 11.1 s


In [1]:
import pandas as pd
data = pd.read_csv("./data/filtered_cve_data2.csv")
data

,CVE_ID,Description,Device,Product,Vendor,Version,Firmware,Patch Availability,Reserved Date,Published Date,Update Date,Problem Type,EPSS_Score,CVSS_Score,Severity,Vector,Exploitability Score,Impact Score,Score Source,references
0,CVE-2009-3564,puppetmasterd in puppet 0.24.6 does not reset ...,Switch,NaN,NaN,NaN,NaN,NaN,2009-10-05T00:00:00,2009-10-06T17:22:00,2024-08-07T06:31:10.575Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CVE-2009-3341,Buffer overflow on the Linksys WRT54GL wireles...,Router,NaN,NaN,NaN,NaN,NaN,2009-09-24T00:00:00Z,2009-09-24T16:00:00Z,2024-09-17T03:03:10.465Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-2009-3962,The management interface on the 2wire Gateway ...,Router,NaN,NaN,NaN,NaN,NaN,2009-11-17T00:00:00,2009-11-17T18:00:00,2024-08-07T06:45:50.747Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CVE-2009-3828,The web interface for Everfocus EDR1600 DVR al...,NVR,NaN,NaN,NaN,NaN,NaN,2009-10-30T00:00:00,2009-10-30T19:00:00,2024-08-07T06:38:30.497Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CVE-2009-3322,The Siemens Gigaset SE361 WLAN router allows r...,Router,NaN,NaN,NaN,NaN,NaN,2009-09-23T00:00:00,2009-09-23T10:00:00,2024-08-07T06:22:24.435Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7398,CVE-2024-23910,Cross-site request forgery (CSRF) vulnerabilit...,Router,WRC-1167GS2-B,"ELECOM CO.,LTD.",v1.67 and earlier,NaN,NaN,2024-02-15T01:25:06.163Z,2024-02-28T23:07:02.324Z,2024-09-09T06:36:00.982Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7399,CVE-2024-23727,The YI Smart Kami Vision com.kamivision.yismar...,Camera,NaN,NaN,NaN,NaN,NaN,2024-01-21T00:00:00,2024-03-28T00:00:00,2024-08-27T19:12:26.981Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7400,CVE-2024-23842,Improper Input Validation in Hitron Systems DV...,NVR,DVR LGUVR-16H,Hitron Systems DVR,1.02,NaN,NaN,2024-01-23T04:53:48.120Z,2024-01-23T04:56:41.242Z,2024-10-22T03:55:45.527Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7401,CVE-2024-23614,A buffer overflow vulnerability exists in Syma...,Router,Messaging Gateway,Symantec,0,['Linux'],NaN,2024-01-18T21:37:15.392Z,2024-01-25T23:32:21.154Z,2024-09-05T18:28:07.954Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
